# Object Tracking with Siamese Networks

In this session we will train a siamese network to optimize the euclidean distance between positive and negative image pairs. This will serve as a toy example of a tracking scenario, in which we need to match a set of detected objects in two consecutive frames. We will use MNIST dataset to train this network.

In [1]:
from __future__ import absolute_import
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

import random
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Lambda
from keras.optimizers import RMSprop
from keras import backend as K

Using TensorFlow backend.


Here we define the loss function that we want to use, which uses the euclidean distance as metric. The loss is defined so that it minimizes the distance between positive pairs and maximizes it for negative pairs, with a certain margin.

In [2]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

We need to define the pairs of positives and negatives for training. A positive pair will be compose of two samples that belong to the same category. The function ```create_pairs``` will do this for us:

In [3]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    
    # number of samples per category
    n = min([len(digit_indices[d]) for d in range(10)]) - 1
    for d in range(10):
        for i in range(n):
            # z1 are positive pairs
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            # add the samples of z1 and x2 indices to pairs
            pairs += [[x[z1], x[z2]]]
            # select random sample from another category
            inc = random.randrange(1, 10)
            dn = (d + inc) % 10
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

Here we creat a simple architecture. This will be shared between the two samples in the pairs.

In [4]:
def create_base_network(input_dim):
    '''Base network to be shared (eq. to feature extraction).
    '''
    seq = Sequential()
    seq.add(Dense(128, input_shape=(input_dim,), activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    seq.add(Dropout(0.1))
    seq.add(Dense(128, activation='relu'))
    return seq

In [7]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
input_dim = 784
nb_epoch = 20

# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(10)]
tr_pairs, tr_y = create_pairs(X_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(10)]
te_pairs, te_y = create_pairs(X_test, digit_indices)

# network definition
base_network = create_base_network(input_dim)

input_a = Input(shape=(input_dim,))
input_b = Input(shape=(input_dim,))

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model = Model(input=[input_a, input_b], output=distance)

# train
rms = RMSprop()
model.compile(loss=contrastive_loss, optimizer=rms)
model.fit([tr_pairs[:, 0], tr_pairs[:, 1]], tr_y,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y),
          batch_size=128,
          nb_epoch=nb_epoch,
          verbose=2)

Train on 108400 samples, validate on 17820 samples
Epoch 1/20
7s - loss: 0.0949 - val_loss: 0.0432
Epoch 2/20
4s - loss: 0.0385 - val_loss: 0.0293
Epoch 3/20
4s - loss: 0.0271 - val_loss: 0.0259
Epoch 4/20
4s - loss: 0.0213 - val_loss: 0.0245
Epoch 5/20
4s - loss: 0.0181 - val_loss: 0.0226
Epoch 6/20
4s - loss: 0.0163 - val_loss: 0.0222
Epoch 7/20
4s - loss: 0.0147 - val_loss: 0.0220
Epoch 8/20
4s - loss: 0.0135 - val_loss: 0.0217
Epoch 9/20
4s - loss: 0.0125 - val_loss: 0.0211
Epoch 10/20
4s - loss: 0.0122 - val_loss: 0.0213
Epoch 11/20
4s - loss: 0.0114 - val_loss: 0.0221
Epoch 12/20
4s - loss: 0.0108 - val_loss: 0.0201
Epoch 13/20
4s - loss: 0.0102 - val_loss: 0.0211
Epoch 14/20
4s - loss: 0.0098 - val_loss: 0.0223
Epoch 15/20
4s - loss: 0.0096 - val_loss: 0.0227
Epoch 16/20
4s - loss: 0.0092 - val_loss: 0.0220
Epoch 17/20
4s - loss: 0.0090 - val_loss: 0.0225
Epoch 18/20
4s - loss: 0.0085 - val_loss: 0.0215
Epoch 19/20
4s - loss: 0.0084 - val_loss: 0.0225
Epoch 20/20
4s - loss: 0.00

In [ ]:
# compute final accuracy on training and test sets
pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(pred, tr_y)
pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(pred, te_y)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))